In [ ]:
# Parameters
model_classic = None
model_sync = None
model_async = None
include_classification_report = None

In [ ]:
from IPython.display import display, HTML

display(HTML("<style>.container { width:80% !important; }</style>"))

from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import subprocess
import os


def format_timedelta(x, pos):
    td = timedelta(seconds=x)
    minutes, remainder = divmod(td.seconds, 60)
    seconds = remainder
    milliseconds = td.microseconds // 1000
    return f"{minutes:02d}:{seconds:02d}:{milliseconds:03d}"

# Plots

## Classic SGD (non distributed)

In [ ]:
file_name = "log.log"
if (
    os.path.exists(file_name) and model_classic is not None
):  # remove model_classic condition if run from notebook
    lines = []
    with open("log.log", "r") as log_file:
        # Iterate through each line in the log file
        for line in log_file:
            lines.append(line.split(" - __main__ - "))

    for i, line in enumerate(lines):
        timestamp = datetime.strptime(line[0], "%Y-%m-%d %H:%M:%S,%f")
        lines[i][0] = timestamp

    del_list = []
    for i, line in enumerate(lines):
        if "INFO" in line[1]:
            del_list.append(i)

    del_list = sorted(del_list, reverse=True)
    for idx in del_list:
        if idx < len(lines):
            lines.pop(idx)

    start_time = lines[0][0]
    end_time = lines[-1][0]
    training_time_classic = end_time - start_time

    minutes, remainder = divmod(training_time_classic.seconds, 60)
    seconds = remainder
    milliseconds = training_time_classic.microseconds // 1000
    formatted_training_time_classic = f"{minutes:02d}:{seconds:02d}:{milliseconds:03d}"
    print("Classic training time:", formatted_training_time_classic)  # MM:SS:sss

    for i, line in enumerate(lines):
        lines[i][0] = line[0] - start_time
        lines[i][1] = line[1].split("DEBUG - ")[1].strip()

    model_update_lines = []
    for line in lines:
        splited_text = line[1].split(",")
        model_update_lines.append(
            (
                line[0],
                float(splited_text[0].split(" ")[1]),
                int(splited_text[1].split(" ")[3][1:-1].split("/")[0]),
                int(splited_text[2].split(" ")[2].split("/")[0]),
            )
        )

    ####### PLOTS #######
    # Create a 1x2 grid of subplots
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20, 5))

    # First subplot (Model Loss vs Time)
    timedeltas = [line[0].total_seconds() for line in model_update_lines]
    losses = [line[1] for line in model_update_lines]

    axs[0].scatter(timedeltas, losses, marker="o")
    axs[0].set_xlabel("Time (MM:SS:sss)")
    axs[0].set_ylabel("Model Loss")
    axs[0].set_title("Classic SGD evolution of the model loss in function of time")

    # Format x-axis tick labels
    formatter = FuncFormatter(format_timedelta)
    axs[0].xaxis.set_major_formatter(formatter)

    # Second subplot (Cumulative Batch Update Count vs Time)
    batches = [line[2] for line in model_update_lines]

    axs[1].plot(timedeltas, batches)

    axs[1].set_xlabel("Time (MM:SS:sss)")
    axs[1].set_ylabel("Cumulative Batch Update Count")
    axs[1].set_title("Classic SGD computation speed")

    # Format x-axis tick labels
    axs[1].xaxis.set_major_formatter(formatter)

    # Display the subplots
    plt.show()

## Synchronous SGD

In [ ]:
file_name = "log_sync.log"
if os.path.exists(file_name):
    lines = []
    with open("log_sync.log", "r") as log_file:
        # Iterate through each line in the log file
        for line in log_file:
            lines.append(line.split(" - __mp_main__ - "))

    for i, line in enumerate(lines):
        timestamp = datetime.strptime(line[0], "%Y-%m-%d %H:%M:%S,%f")
        lines[i][0] = timestamp

    nb_workers = int(
        (lines[0][1].split("with ")[1]).split(" workers")[0]
    )  # extract from first line

    del_list = []
    for i, line in enumerate(lines):
        if "INFO" in line[1]:
            del_list.append(i)

    del_list = sorted(del_list, reverse=True)
    for idx in del_list:
        if idx < len(lines):
            lines.pop(idx)

    start_time = lines[0][0]
    end_time = lines[-1][0]
    training_time_sync = end_time - start_time

    minutes, remainder = divmod(training_time_sync.seconds, 60)
    seconds = remainder
    milliseconds = training_time_sync.microseconds // 1000
    formatted_training_time_sync = f"{minutes:02d}:{seconds:02d}:{milliseconds:03d}"
    print("Synchronous training time:", formatted_training_time_sync)  # MM:SS:sss

    for i, line in enumerate(lines):
        lines[i][0] = line[0] - start_time
        lines[i][1] = line[1].split("DEBUG - ")[1].strip()

    model_loss_lines = []
    worker_update_lines = []
    for i, line in enumerate(lines):
        if i < nb_workers:
            continue
        if "PS updated model, " in line[1]:
            # timedetla, model loss
            model_loss_lines.append((line[0], float(line[1].split("is ")[1])))
        elif "PS got " in line[1]:
            splited_text = (line[1].split("from ")[1]).split(" ")
            # timedelta, worker id, batch count, epoch
            worker_update_lines.append(
                (
                    line[0],
                    int(splited_text[0].split("_")[1][:-1]),
                    int(splited_text[2].split("/")[0][1:]),
                    int(splited_text[-1].split("/")[0]),
                )
            )

    ####### PLOTS #######
    # Create a 1x2 grid of subplots
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20, 5))

    # First subplot (Model Loss vs Time)
    timedeltas = [line[0].total_seconds() for line in model_loss_lines]
    losses = [line[1] for line in model_loss_lines]

    axs[0].scatter(timedeltas, losses, marker="o")
    axs[0].set_xlabel("Time (MM:SS:sss)")
    axs[0].set_ylabel("Model Loss")
    axs[0].set_title("Synchronous SGD evolution of the model loss in function of time")

    # Format x-axis tick labels
    formatter = FuncFormatter(format_timedelta)
    axs[0].xaxis.set_major_formatter(formatter)

    # Second subplot (Cumulative Batch Update Count vs Time)
    worker_cumulative_updates = {}
    for line in worker_update_lines:
        td, worker_id, batch_count, epoch = line
        worker_cumulative_updates.setdefault(worker_id, []).append((td, batch_count))

    for worker_id, updates in worker_cumulative_updates.items():
        x = [td.total_seconds() for td, _ in updates]
        y = [batch_count for _, batch_count in updates]
        axs[1].plot(x, y, label=f"Worker {worker_id}")

    axs[1].set_xlabel("Time (MM:SS:sss)")
    axs[1].set_ylabel("Cumulative Batch Update Count")
    axs[1].set_title("Synchronous SGD workers speed comparison")
    axs[1].legend()

    # Format x-axis tick labels
    axs[1].xaxis.set_major_formatter(formatter)

    # Display the subplots
    plt.show()

## Asynchronous SGD

In [ ]:
file_name = "log_async.log"
if os.path.exists(file_name):
    lines = []
    with open("log_async.log", "r") as log_file:
        # Iterate through each line in the log file
        for line in log_file:
            lines.append(line.split(" - __mp_main__ - "))

    for i, line in enumerate(lines):
        timestamp = datetime.strptime(line[0], "%Y-%m-%d %H:%M:%S,%f")
        lines[i][0] = timestamp

    nb_workers = int(
        (lines[0][1].split("with ")[1]).split(" workers")[0]
    )  # extract from first line

    del_list = []
    for i, line in enumerate(lines):
        if "INFO" in line[1]:
            del_list.append(i)

    del_list = sorted(del_list, reverse=True)
    for idx in del_list:
        if idx < len(lines):
            lines.pop(idx)

    start_time = lines[0][0]
    end_time = lines[-1][0]
    training_time_async = end_time - start_time

    minutes, remainder = divmod(training_time_async.seconds, 60)
    seconds = remainder
    milliseconds = training_time_async.microseconds // 1000
    formatted_training_time_async = f"{minutes:02d}:{seconds:02d}:{milliseconds:03d}"
    print("Asynchronous training time:", formatted_training_time_async)  # MM:SS:sss

    for i, line in enumerate(lines):
        lines[i][0] = line[0] - start_time
        lines[i][1] = line[1].split("DEBUG - ")[1].strip()

    worker_update_lines = []
    for i, line in enumerate(lines):
        if i < nb_workers:
            continue
        if "PS updated model, worker loss: " in line[1]:
            splited_text = (line[1].split("PS updated model, worker loss: "))[1].split(
                " "
            )
            worker_update_lines.append(
                (
                    line[0],
                    float(splited_text[0]),
                    int(splited_text[1].split("_")[1][:-1]),
                )
            )

    ####### PLOTS #######
    # Create a 1x2 grid of subplots
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(20, 5))

    # First subplot (Worker Loss vs Time)
    worker_losses = {}
    for line in worker_update_lines:
        td, worker_loss, worker_id = line
        worker_losses.setdefault(worker_id, []).append((td, worker_loss))

    for worker_id, losses in worker_losses.items():
        x = [td.total_seconds() for td, _ in losses]
        y = [worker_loss for _, worker_loss in losses]
        axs[0].scatter(x, y, label=f"Worker {worker_id}", marker="o")

    axs[0].set_xlabel("Time (MM:SS:sss)")
    axs[0].set_ylabel("Worker Loss")
    axs[0].set_title("Asynchronous SGD evolution of worker loss in function of time")
    axs[0].legend()

    # Format x-axis tick labels
    formatter = FuncFormatter(format_timedelta)
    axs[0].xaxis.set_major_formatter(formatter)

    # Second subplot (Cumulative Batch Update Count vs Time)
    worker_cumulative_updates = {}
    for line in worker_update_lines:
        td, _, worker_id = line
        worker_cumulative_updates.setdefault(worker_id, []).append(td)

    for worker_id, updates in worker_cumulative_updates.items():
        x = [td.total_seconds() for td in updates]
        y = list(range(1, len(updates) + 1))
        axs[1].plot(
            x,
            y,
            label=f"Worker {worker_id}",
        )

    axs[1].set_xlabel("Time (MM:SS:sss)")
    axs[1].set_ylabel("Cumulative Batch Update Count")
    axs[1].set_title("Asynchronous SGD workers speed comparison")
    axs[1].legend()

    # Format x-axis tick labels
    axs[1].xaxis.set_major_formatter(formatter)

    # Display the subplots
    plt.show()

# Train/Test Performance

## Classic SGD

In [ ]:
# !python test_model.py mnist_classic_05_0001_00_32_2.pt
if model_classic is not None:
    if include_classification_report is not None:
        subprocess.run(
            ["python3", "test_model.py", model_classic, "--classification_report"]
        )
    else:
        subprocess.run(["python3", "test_model.py", model_classic])

## Synchronous SGD

In [ ]:
# !python test_model.py mnist_sync_3_05_0001_00_32_2.pt
if model_sync is not None:
    if include_classification_report is not None:
        subprocess.run(
            ["python3", "test_model.py", model_sync, "--classification_report"]
        )
    else:
        subprocess.run(["python3", "test_model.py", model_sync])

## Asynchronous SGD

In [ ]:
# !python3 test_model.py mnist_async_3_05_0001_00_32_2.pt
if model_async is not None:
    if include_classification_report is not None:
        subprocess.run(
            ["python3", "test_model.py", model_async, "--classification_report"]
        )
    else:
        subprocess.run(["python3", "test_model.py", model_async])